In [ ]:
# making full json config from {'pathIn' : '...'}
def make_full_json_config():
    import os
    import simplejson as json

    with open('1.json') as data_file:    
        data = json.load(data_file)
    dirs = [d for d in os.listdir(data['pathIn']) if os.path.isdir(os.path.join(data['pathIn'], d))]
    data['scenes'] = []
    id = 0
    for d in dirs:
        tmp = {}
        tmp['id'] = id
        id += 1
        tmp['path'] = d
        tmp['bit'] = '16' if (d[2] == '8') else '8'
        data['scenes'].append(tmp)
    with open('1.json', 'w') as outfile:
        json.dump(data, outfile, sort_keys = True, indent = 4, ensure_ascii = False)
        
# copy all need files from Landsats_done
def copy_need_files():
    str = "/media/fila/DATA/landsat/Landsats_done"
    str1 = "/media/fila/DATA/landsat/in"
    with open('1.json') as data_file:    
        data = json.load(data_file)
    for i in data['scenes']:
        file1 = os.path.join(str, i['path'], i['path'] + data['suffs']['in'][i['bit']]['red'])
        file2 = os.path.join(str, i['path'], i['path'] + data['suffs']['in'][i['bit']]['nir'])
        os.system("cp " + file1 + " " + str1)
        os.system("cp " + file2 + " " + str1)
    
copy_need_files()

In [10]:
import os
import simplejson as json
import numpy as np
import skimage.io
import matplotlib
import scipy.spatial.distance

# class for one scene analyse
class Scene:
    def __init__(self, _id_):
        self.id = _id_
        with open('1.json') as data_file:    
            self.json = json.load(data_file)
        self.bit = self.json['scenes'][self.id]['bit']
        self.path = self.json['scenes'][self.id]['path']
            
    def is_in_mask(self, img, x, y):
        if self.bit == '16':
            return img[x, y] < 64000 # strange constant
        else:
            return img[x, y] < 253
        
    def get_white_pixel(self):
        if self.bit == '16':
            return 65535
        else:
            return 255
    
    # type from  ['in', 'inFiltered', 'solp']
    def read_images(self, type):
        if (type not in ['in', 'inFiltered', 'solp']):
            raise 'read_image: wrong type'
        if (type in ['in', 'inFiltered']):
            file_red = os.path.join(self.json['pathIn'], self.path + self.json['suffs'][type][self.bit]['red'])
            file_nir = os.path.join(self.json['pathIn'], self.path + self.json['suffs'][type][self.bit]['nir'])
        else:
            file_red = os.path.join(self.json['pathOut'], self.json['pathSolp'], self.path + 
                                    self.json['suffs']['in'][self.bit]['red'])
            file_nir = os.path.join(self.json['pathOut'], self.json['pathSolp'], self.path + 
                                    self.json['suffs']['in'][self.bit]['nir'])
        return {'red': skimage.io.imread(file_red), 'nir': skimage.io.imread(file_nir)}
    
    def get_plot_array(self, img_red, img_nir):
        sizes = self.json['plotSizes'][self.bit]
        plot_array = np.zeros([sizes['red'][1] - sizes['red'][0] + 1,
                               sizes['nir'][1] - sizes['nir'][0] + 1], dtype = 'int32')
        for x, y in np.ndindex(img_red.shape):
            if self.is_in_mask(img_red, x, y) and self.is_in_mask(img_nir, x, y) \
                and sizes['red'][0] <= img_red[x, y] <= sizes['red'][1] \
                and sizes['nir'][0] <= img_nir[x, y] <= sizes['nir'][1]:
                    plot_array[img_nir[x, y] - sizes['nir'][0], img_red[x, y] - sizes['red'][0]] += 1
        return plot_array
    
    def save_plot_array(self, plot_array):
        matplotlib.image.imsave(os.path.join(self.json['pathOut'], self.json['pathNotMarkedPlots'], 
                                             self.path + self.json['suffs']['markedPlots']), 
                                plot_array[::-1, :])
        
    def make_solp_scene(self):
        imgs = self.read_images('inFiltered')
        sizes = self.json['plotSizes'][self.bit]
        plot_marked = skimage.io.imread(os.path.join(self.json['pathOut'], self.json['pathMarkedPlots'], 
                                                     self.path + self.json['suffs']['markedPlots']))
        colors = self.json['colorsMarkedPlots'][1:2:1]
        print colors
        color_eps = 9
        __pixels__ = 0
        for x, y in np.ndindex(imgs['red'].shape):
            red, nir = imgs['red'][x, y], imgs['nir'][x, y]
            if not sizes['red'][0] <= red <= sizes['red'][1] or not sizes['nir'][0] <= nir <= sizes['nir'][1]:
                for img in imgs.values():
                    img[x, y] = self.get_white_pixel()
                continue
            for color in colors:
                if scipy.spatial.distance.pdist([plot_marked[sizes['nir'][1] - nir, red - sizes['red'][0]][0:3:1],
                                                 color], 'minkowski', 1)[0] < color_eps:
                    __pixels__ += 1
                    break
            else:
                for img in imgs.values():
                    img[x, y] = self.get_white_pixel()
        tmp_arr = ['red', 'nir']
        for tmp in tmp_arr:
            skimage.io.imsave(os.path.join(self.json['pathOut'], self.json['pathSolp'], 
                                           self.path + self.json['suffs']['in'][self.bit][tmp]), 
                              imgs[tmp], plugin = 'tifffile')
        print __pixels__
    
    def get_soil_line(self):
        imgs = self.read_images('solp')
        plot_array = A.get_plot_array(imgs['red'], imgs['nir'])
        red, nir = [], []
        for x, y in np.ndindex(imgs['red'].shape):
            red.append(imgs['red'][x, y])
            nir.append(imgs['nir'][x, y])
        red_matrix = np.vstack([red, np.ones(len(red))]).T
        return np.linalg.lstsq(red_matrix, nir)[0]
        
        
           
        
A = Scene(3)
A.make_solp_scene()
#A.get_soil_line()
#imgs = A.read_images('in')
#plot_array = A.get_plot_array(imgs['red'], imgs['nir'])
#A.save_plot_array(plot_array)


[[100, 255, 100]]
361028


In [ ]:
A.make_solp_scene()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
plt.imshow(plot_array[::-1, :])
#plt.xlim([0, 255])
#plt.ylim([255, 0])
#plt.axis('off')
#ax = plt.gca()
#ax.xaxis.set_major_formatter(plt.NullFormatter())
#ax.yaxis.set_major_formatter(plt.NullFormatter())
c = plt.gcf()
#plt.gcf().set_size_inches(1, 1)
#c.subplots_adjust(bottom=0.,left=0.,right=1.,top=1.)


#plt.autoscale(True, 'both')

plt.axes().axes.get_yaxis().set_visible(False)
plt.axes().axes.get_xaxis().set_visible(False)


c = plt.gcf()
c.set_size_inches(1., 1.)
#c.set_dpi(256 / 2.00)

plt.savefig('test2.png', dpi=256, bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:
import matplotlib
matplotlib.image.imsave('name.png', plot_array[::-1, :])

In [ ]:
str = "/media/fila/DATA/landsat/Landsats_done"
str1 = "/media/fila/DATA/landsat/in"
with open('1.json') as data_file:    
    data = json.load(data_file)
for i in data['scenes']:
    file1 = os.path.join(str, i['path'], i['path'] + i['suffNir'])
    file2 = os.path.join(str, i['path'], i['path'] + i['suffRed'])
    os.system("cp " + file1 + " " + str1)
    os.system("cp " + file2 + " " + str1)

In [ ]:
img = skimage.io.imread('/media/fila/DATA/landsat/in/LC81780232014088LGN00_B4_C_F.tif')
print img[[0,0]]

In [ ]:
for x, y in np.ndindex(img.shape[0:2:1]):
    p = img[x, y]
    if abs(p[0] - 100) <= 5 and abs(p[1] - 255) <= 5 and abs(p[2] - 100) <= 5:
        print p

In [ ]:
img.shape[0:2:1]

In [ ]:
for i in range(5):
    for j in range(5):
        break
    else:
        print '111'

In [ ]:
a = {'red': 123, 'nir': 234}
for aaa in a.values():
    print aaa

In [6]:
qqq = np.array([1,2])
print qqq[1:2:1]

[2]
